<a href="https://colab.research.google.com/github/geoaura/KSAFM_Data_FloX/blob/main/Data_process_dp_10min_RDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###########################################
#엑셀 자료 처리 python code
###########################################
#Input: excel
#Output: csv
###########################################
#Code update info.
#v01.0, 22.07.08., made by Jae-Hyun RYU
#       Read, Process, and Write code
###########################################

# 라이브러리 읽기

In [8]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

#설정

In [10]:
input_data ='/content/drive/MyDrive/Colab_Notebooks/RDA/LineQ/0706_CR1000X_gj1_GJ7_10min_2022_06_22_15_30_57.xlsx'
output_path='/content/drive/MyDrive/Colab_Notebooks/RDA/LineQ/0706_CR1000X_gj1_GJ7_10min_2022_06_22_15_30_57_ryu.csv'
set_first_date='2022-06-10 11:00:00'
set_end_date  ='2022-06-22 15:30:00'
set_time_freq ='10min'

#자료 읽기

In [11]:
column_names=['date','time','water_level_mm','Batt_volt_Min','PTemp']
dataset=pd.read_excel(input_data,skiprows=1)[column_names]
dataset['water_level_mm']=pd.to_numeric(dataset['water_level_mm'], errors='coerce')
dataset['Batt_volt_Min'] =pd.to_numeric(dataset['Batt_volt_Min'] , errors='coerce')
dataset['PTemp']         =pd.to_numeric(dataset['PTemp']         , errors='coerce')

#날짜와 시간 합치기

In [12]:
merge_datetime = [datetime.combine(a,b) for a, b in zip(dataset['date'], dataset['time'])]
dataset.insert(0,"datetime",merge_datetime, True)

# 시간 간격 설정

In [13]:
first_date= datetime.strptime(set_first_date,'%Y-%m-%d %H:%M:%S')
end_date  = datetime.strptime(set_end_date  ,'%Y-%m-%d %H:%M:%S')
refTT      = pd.date_range(first_date, end_date, freq=set_time_freq)

#설정된 시간 간격에 따라 자료 처리

In [14]:
Xaxis_date=[]
water=[]
batt =[]
temp =[]
for dn in refTT[:]:
  first_hour=datetime.strftime(dn,'%Y-%m-%d %H:%M:%S') 
  end_hour  =datetime.strptime(first_hour,'%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)

  dataset_filter=dataset.copy()
  dataset_filter=dataset_filter[(dataset_filter['datetime']>first_hour) & (dataset_filter['datetime']<=end_hour)]

  water_value=pd.to_numeric(dataset_filter['water_level_mm'], errors='coerce')
  batt_value =pd.to_numeric(dataset_filter['Batt_volt_Min'] , errors='coerce')
  temp_value =pd.to_numeric(dataset_filter['PTemp']         , errors='coerce')

  Xaxis_date.append( datetime.strftime(end_hour,'%Y-%m-%d %H:%M') )

  if len(water_value)>0:
    water.append(round(np.nanmedian(water_value),4))
    batt.append(round(np.nanmin(batt_value),4))
    temp.append(round(np.nanmedian(temp_value),4))
  else:
    water.append(-999)
    batt.append(-999)
    temp.append(-999)

#자료 저장

In [15]:
data_array=list(zip(Xaxis_date, water, batt, temp))
save_data_array=pd.DataFrame(data_array)
save_data_array=save_data_array.drop_duplicates()

column_names=['datetime','water_level_mm','Batt_volt_Min','PTemp']
save_data_array.to_csv(output_path, sep=',', na_rep='-999', header=column_names,  index=False) 